# Our Quantum Simulator
## Group: Brandon, Chris, Jacob, Michael

In [32]:
import gates
import utilities
import random


In [33]:
class QuantumProgram():
    def __init__(self,the_number_of_qubits):
        self.qubits = gates.zeroQubit()
        self.qubit_number = the_number_of_qubits
        self.operations = []
        self.classical = []
        for i in range(1, the_number_of_qubits):
            self.qubits = utilities.tensor(self.qubits,gates.zeroQubit())
    def getQubits(self):
        return self.qubits
    def addQubits(self, the_number_of_qubits):
        self.qubit_number += the_number_of_qubits
        for i in range(0, the_number_of_qubits):
            self.qubits = utilities.tensor(self.qubits,gates.zeroQubit())
    def getOperations(self):
        return self.operations
    def getNumberOfOperations(self):
        return len(self.operations)
    def measureOutcome(self,binNum):
        #000 ->0
        #001 ->1
        #010 ->2
        #011 ->3
        zeroRange = self.observing_probabilities()[binNum]
        random.seed()
        measurement = random.random()
        if measurement >= zeroRange:
            measurement = 0
        else:
            measurement = 1
            
        return measurement
    def measureMultiple(self):#Pick a random category and measure it
        probabilities = self.observing_probabilities()
        while True:
            binNum = random.randint(0,len(probabilities)-1)
            if(self.measureOutcome(binNum)==1):
                return binNum

    def gateMaker(self,position_of_qubit,gateFunction):
        (self.qubits,returnedGate)=gates.apply_gate(gateFunction,self.qubits,position_of_qubit)
        self.operations.append(returnedGate)
        
    def H(self, *args): #Shorthand common gate calls for easier end-user coding
        if len(args) > 0:
            self.gateMaker(args[0],gates.H())
        else:
            return gates.H()
    def NOT(self, *args):
        if len(args) > 0:
            self.gateMaker(args[0],gates.NOT())
        else:
            return gates.NOT()
    def X(self, *args):
        if len(args) > 0:
            self.gateMaker(args[0],gates.NOT())
        else:
            return gates.NOT()
    def Z(self, *args):
        if len(args) > 0:
            self.gateMaker(args[0],gates.Z())
        else:
            return gates.Z()
        
    def Control(self,gateFunction,position_of_qubit,controls=[]):#Gate function being one of the above gates
        mat2=[]
        gateReturn = []
        mat2,gateReturn=gates.apply_gate(gateFunction,self.qubits,position_of_qubit,controls)
        self.qubits = mat2
        self.operations.append(gateReturn)
        return mat2
    def ControlClassic(self,gateFunction,position_of_qubit,controls=[],theta=None):#Gate function being one of the above gates
        mat2=[]
        gateReturn = []
        if 0 not in controls:
            if theta != None:
                mat2,gateReturn=gates.apply_gate(gateFunction, self.qubits,position_of_qubit)
            else: 
                mat2,gateReturn=gates.apply_gate(gateFunction,self.qubits,position_of_qubit)
        else:
            mat2,gateReturn=gates.apply_gate(gates.I(),self.qubits,position_of_qubit)
        self.qubits = mat2
        self.operations.append(gateReturn)
        return mat2

    def read_unitary(self):
        resultant = self.operations[0]
        first = True
        for operation in self.operations:
            if first:
                first = False
                continue
            else:
                resultant = utilities.matmul(resultant,operation)
        return resultant
    def read_state(self):
        return self.qubits
    def observing_probabilities(self):
        probabilities = []
        for i in range(0, len(self.qubits[0])):
            probabilities.append(self.qubits[0][i]*self.qubits[0][i])
        return probabilities
    def execute(self,the_number_of_shots):
        executionDictionary = {}
        self.observing_probabilities()
        for string in utilities.getBinaryStrings(self.qubit_number):
            executionDictionary.setdefault(string,0)
        
        for i in range(0,the_number_of_shots):
            intNum =self.measureMultiple()
            binNumSize= self.qubit_number
            formatter= ("0"+str(binNumSize)+'b')
            binNum = format(intNum,formatter)
            executionDictionary[binNum] += 1  
        return executionDictionary
            
        

In [34]:
#Simple Hadamard on the first qubit example
qc = QuantumProgram(3)
test = qc.getQubits()
qc.gateMaker(0,qc.H())

probabilities= qc.observing_probabilities()
results = qc.execute(50)
print(results)

{'000': 26, '001': 0, '010': 0, '011': 0, '100': 24, '101': 0, '110': 0, '111': 0}


In [35]:
#Superdense encoding example
encoder = QuantumProgram(2)

#Entangle
encoder.H(0)
encoder.Control(encoder.X(),1,[0])

#Encode with:
bit1 = 1
bit2 = 0
encoder.ControlClassic(encoder.X(),0,[bit2])
encoder.ControlClassic(encoder.Z(),0,[bit1])

#Decode
encoder.Control(encoder.X(),1,[0])
encoder.H(0)

print(encoder.observing_probabilities())
print(encoder.execute(50))

[0.0, 0.0, 1.0000000000000004, 0.0]
{'00': 0, '01': 0, '10': 50, '11': 0}
